In [4]:
import pandas as pd
import numpy as np
import random
import math

In [5]:
# Branje ocen
class UserItemData:

    #data from table
    #konstruktor

    def __init__(self, path, from_date=None, to_date=None, min_ratings=0):
        self.df = pd.read_csv(path, "\t", encoding="ISO-8859-1")

        #split from date to day, month, year
        if (from_date != None):
            from_date_a = from_date.split('.')
            to_date_a = to_date.split('.')
            # format of date is like that year-month-day
            from_date_datef = str(f'{from_date_a[2]}-{from_date_a[1]}-{from_date_a[0]}')
            to_date_datef = str(f'{to_date_a[2]}-{to_date_a[1]}-{to_date_a[0]}')
            #make new column dates
            self.df['dates'] = self.df["date_year"].astype(str).str.cat(self.df[['date_month', 'date_day']].astype(str), sep='-')
            self.df['dates'] = pd.to_datetime(self.df['dates'])
            self.df = self.df[self.df['dates'].between(from_date_datef, to_date_datef, inclusive=True)]

        self.df = self.df.groupby("movieID").filter(lambda ratings: len(ratings) >= min_ratings)

    def nratings(self):
        return len(self.df['rating'].values)

    def get_watched_movie_list(self, userId):
        return self.df[self.df["userID"] == userId]['movieID'].values
    
    def get_all_movies_id(self):
        return self.df['movieID'].values

    #vs - vsota vseh ocen za film movieId
    def get_sum_rating_movie(self, movieId):
        return sum(self.df[self.df['movieID'] == movieId]['rating'])
   

    def get_sum_rating_all_movies(self):
        return sum(self.df['rating'])
    
    # n - stevilo ocen ki jih je dobil film
    def get_number_rating_movie(self, movieId):
        return len(self.df[self.df['movieID'] == movieId])

    def get_rating_of_movie(self, movieId):
        return list(self.df[self.df['movieID'] == movieId]['rating'].values)

    def get_all_users(self):
        return list(set(self.df['userID']))

    def get_number_users_rated_movies(self, movie_id1, movie_id2):
        return len(self.df[(self.df['movieID'] == movie_id1) | (self.df['movieID'] == movie_id2)]['userID'])

    def return_numpy_df(self):
        return self.df[['movieID', 'userID', 'rating']].to_numpy()

    def get_rating_movie(self, user_id, movie_id):
        return self.df[(self.df['userID'] == user_id) & (self.df['movieID'] == movie_id)]['rating'].values


In [14]:
#Branje ocen

In [6]:
uim = UserItemData('data/user_ratedmovies.dat')
print(uim.nratings())

uim = UserItemData('data/user_ratedmovies.dat', from_date = '12.1.2007', to_date='16.2.2008', min_ratings=100)
print(uim.nratings())

855598
73657


In [7]:
class MovieData:
    def __init__(self, path):
        # encoding not working on mac air so thats why encoiding is set
        self.df = pd.read_csv(path, sep='\t', encoding='ISO-8859-1')

    def get_title(self, movieID):
        data = self.df[self.df.id == movieID]['title']
        return data.values[0]

In [17]:
# Branje filmov

In [8]:
md = MovieData('data/movies.dat')
print(md.get_title(1))

Toy story


In [21]:
# Nakljucni predikator

In [9]:
class RandomPredictor:
    def __init__(self, min, max):
        self.min = min
        self.max = max

    def predict(self, user_id):
        user_ratings = dict()
        #min_movieID = self.user_item_data.df["movieID"].min()
        #max_movieID = self.user_item_data.df["movieID"].max()
        list_of_movieIDs = list(set(self.user_item_data.get_all_movies_id()))
        list_of_movieIDs.sort()
        #print('list of ids ', list_of_movieIDs)
        for movieID in list_of_movieIDs:
            user_ratings[movieID] = random.randint(self.min, self.max)
        return user_ratings

    def fit(self, user_item_data):
        self.user_item_data = user_item_data


In [61]:
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = RandomPredictor(1, 5)
rp.fit(uim)
pred = rp.predict(78)
print(type(pred))
items = [1, 3, 20, 50, 100]
for item in items:
    print("Film: {}, ocena: {}".format(md.get_title(item), pred[item]))

<class 'dict'>
Film: Toy story, ocena: 4
Film: Grumpy Old Men, ocena: 3
Film: Money Train, ocena: 3
Film: The Usual Suspects, ocena: 3
Film: City Hall, ocena: 2


In [11]:
class Recommender:
    def __init__(self, predicator):
        self.predicator = predicator
    
    def fit(self, user_item_data):
        self.user_item_data = user_item_data
        self.predicator.fit(self.user_item_data)

    def recommend(self, userID, n=10, rec_seen=True):
        predicted_grade = self.predicator.predict(userID)
        recommendet_movies = []
        watched_movie_list = list(self.user_item_data.get_watched_movie_list(userID))
        if (rec_seen):
            for movieId, grade in predicted_grade.items():
                if movieId in watched_movie_list:
                    recommendet_movies.append((movieId, grade))
        else:
            for movieId, grade in predicted_grade.items():
                if movieId not in watched_movie_list:
                    recommendet_movies.append((movieId, grade))
        #sort recomended movies
        recommendet_movies_sort = sorted(recommendet_movies, key=lambda t: t[1], reverse=True)
        return recommendet_movies_sort[:n]

In [60]:
# Priporocanje z random predict

In [12]:
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = RandomPredictor(1, 5)
rec = Recommender(rp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))  

Film: Father of the Bride Part II, ocena: 5
Film: The American President, ocena: 5
Film: Nixon, ocena: 5
Film: Ace Ventura: When Nature Calls, ocena: 5
Film: Money Train, ocena: 5


In [13]:
class AveragePredictor:
    def __init__(self, b):
        self.b = b

    def fit(self, user_item_data):
        self.user_item_data = user_item_data
        self.g_avg = self.user_item_data.get_sum_rating_all_movies() / self.user_item_data.nratings()

    def calculate(self, movie_id):
        vs = self.user_item_data.get_sum_rating_movie(movie_id)
        n = self.user_item_data.get_number_rating_movie(movie_id)
        return (vs + self.b * self.g_avg) / (n + self.b)


    def predict(self, user_id):
        user_ratings = dict()
        list_of_movieIDs = list(set(self.user_item_data.get_all_movies_id()))
        list_of_movieIDs.sort()
        for movieID in list_of_movieIDs:
            user_ratings[movieID] = self.calculate(movieID)
        return user_ratings

In [59]:
# Napovedovanje s povprecjem

In [14]:
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
av = AveragePredictor(b=100)
rec = Recommender(av)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))  

Film: The Usual Suspects, ocena: 4.225944245560473
Film: The Godfather: Part II, ocena: 4.146907937910189
Film: Cidade de Deus, ocena: 4.116538340205236
Film: The Dark Knight, ocena: 4.10413904093503
Film: 12 Angry Men, ocena: 4.103639627096175


In [15]:
class ViewsPredicator:
    def __init__(self):
        pass

    def fit(self, user_item_data):
        self.user_item_data = user_item_data

    def predict(self, user_id):
        user_ratings = dict()
        list_of_movieIDs = list(set(self.user_item_data.get_all_movies_id()))
        list_of_movieIDs.sort()
        for movieID in list_of_movieIDs:
            user_ratings[movieID] = self.user_item_data.get_number_rating_movie(movieID)
        return user_ratings

In [46]:
# Priporočanje najbolj gledanih filmov

In [16]:
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
vp = ViewsPredicator()
rec = Recommender(vp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val)) 

Film: The Lord of the Rings: The Fellowship of the Ring, ocena: 1576
Film: The Lord of the Rings: The Two Towers, ocena: 1528
Film: The Lord of the Rings: The Return of the King, ocena: 1457
Film: The Silence of the Lambs, ocena: 1431
Film: Shrek, ocena: 1404


In [58]:
# Napovedovanje ocen s podobnostjo med produkti 

In [53]:
class ItemBasedPredictor:
    def __init__(self, min_values=0, threshold=0):
        self.min_values = min_values
        self.threshold = threshold

    def fit(self, user_item_data):
        self.user_item_data = user_item_data
        self.all_users = self.user_item_data.get_all_users()
        self.df_numpy = self.user_item_data.return_numpy_df()
        # get all users and calculates avg rating
        self.user_avg = dict()
        for user in self.all_users:
            user_rating = self.user_item_data.df[self.user_item_data.df['userID'] == user]['rating']
            self.user_avg[user] = sum(user_rating) / len(user_rating)
        self.all_sim = self.calcualte_all_sim_with_numpy()
    
    # calculates all sim for every movie and returns dict 
    # with {(movieId1, movieId2): similarity} 
    def calculate_all_sim(self):
        # all movies and sim
        movie_sim = dict()
        all_movies_np1 = np.unique(self.df_numpy[:, 0])
        all_movies_np2 = np.unique(self.df_numpy[:, 0])
        for movieId1 in all_movies_np1:
            for movieId2 in all_movies_np2:
                if movieId1 != movieId2:
                    movie_sim[(movieId1, movieId2)] = self.similarity(movieId1, movieId2)
        return movie_sim

    def calcualte_all_sim_with_numpy(self):
        movis_sim_np_array = []
        all_movies_np1 = np.unique(self.df_numpy[:, 0])
        all_movies_np2 = np.unique(self.df_numpy[:, 0])
        for movieId1 in all_movies_np1:
            for movieId2 in all_movies_np2:
                if movieId1 != movieId2:
                    similarity = self.similarity(movieId1, movieId2)
                    if similarity > 0:
                        movis_sim_np_array.append([movieId1, movieId2, similarity])
        return np.array(movis_sim_np_array)
    
    
    # for every sim that is >0 get the number of rating that user gave movie
    # formula is sum(sim*rating_user)/sum(sim)
    def predict(self, user_id):
        predict_movie_ratings = dict()
        for movieId in self.all_sim[:, 0]:
            pred_for_user = self.all_sim[np.where(self.all_sim[:, 0] == movieId)]
            formula_first_line = 0
            sum_sim = 0
            i = 0
            for movieid2 in pred_for_user[:, 1]:
                rating = uim.get_rating_movie(user_id, int(movieid2))
                if rating.size > 0:
                    sim = pred_for_user[i, 2]
                    sum_sim += sim
                    formula_first_line += (sim*rating[0])
                i+=1
            pred = formula_first_line/sum_sim

            predict_movie_ratings[movieId] = pred
        return predict_movie_ratings

    def similarity(self, p1, p2):
        #check if number of users that graded two movies are enough
        if (self.user_item_data.get_number_users_rated_movies(p1, p2) < self.min_values):
            return 0.0

        frist_line_in_fromula = 0
        movie1_sqrt = 0
        movie2_sqrt = 0
        movie1_df_filter = self.df_numpy[np.where(self.df_numpy[:, 0] == p1)]
        movie2_df_filter = self.df_numpy[np.where(self.df_numpy[:, 0] == p2)]
        for user in self.user_avg.keys():
            ratings_movie1 = movie1_df_filter[np.where(movie1_df_filter[:, 1] == user)]
            ratings_movie2 = movie2_df_filter[np.where(movie2_df_filter[:, 1] == user)]
            if ratings_movie1.size > 0 and ratings_movie2.size > 0:
                avg_from_user = self.user_avg[user]
                rating_movie1_cal = ratings_movie1[:, -1][0] - avg_from_user
                rating_movie2_cal = ratings_movie2[:, -1][0] - avg_from_user
                frist_line_in_fromula += ((rating_movie1_cal) * (rating_movie2_cal))
                movie1_sqrt += ((rating_movie1_cal) ** 2)
                movie2_sqrt += ((rating_movie2_cal) ** 2)
        self.similarity_result = frist_line_in_fromula / (math.sqrt(movie1_sqrt)*math.sqrt(movie2_sqrt))
        if self.similarity_result < self.threshold:
            return 0.0
        return self.similarity_result
    
        
    
    # item is movieId
    def similarItems(self, item, n):
        movies_np_array = np.unique(self.df_numpy[:, 0])
        most_similar = list()
        for movieId in movies_np_array:
            if movieId != item:
                most_similar.append((movieId, self.similarity(item, movieId)))
        most_similar.sort(key=lambda x : x[1], reverse=True)
        return most_similar[:n]
    
    def return_top20_most_sim(self):
        sorted_sim = self.all_sim[self.all_sim[:, 2].argsort()][::-1]
        return sorted_sim[:20]
    

In [54]:
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat', min_ratings=1000)
rp = ItemBasedPredictor()
rec = Recommender(rp)
rec.fit(uim)
print("Podobnost med filmoma 'Men in black'(1580) in 'Ghostbusters'(2716): ", rp.similarity(1580, 2716))
print("Podobnost med filmoma 'Men in black'(1580) in 'Schindler's List'(527): ", rp.similarity(1580, 527))
print("Podobnost med filmoma 'Men in black'(1580) in 'Independence day'(780): ", rp.similarity(1580, 780))

Podobnost med filmoma 'Men in black'(1580) in 'Ghostbusters'(2716):  0.2339552317675662
Podobnost med filmoma 'Men in black'(1580) in 'Schindler's List'(527):  0.0
Podobnost med filmoma 'Men in black'(1580) in 'Independence day'(780):  0.42466125844687624


In [ ]:
# predict

In [55]:
print("Predictions for 78: ")
rec_items = rec.recommend(78, n=15, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))

Predictions for 78: 
Film: Shichinin no samurai, ocena: 4.3557347903101595
Film: The Usual Suspects, ocena: 4.354681728067835
Film: The Silence of the Lambs, ocena: 4.335305303472517
Film: Sin City, ocena: 4.2786871668991004
Film: Monsters, Inc., ocena: 4.217581136943521
Film: The Incredibles, ocena: 4.207098583281748
Film: The Lord of the Rings: The Fellowship of the Ring, ocena: 4.152792107348346
Film: Batman Begins, ocena: 4.146413806700199
Film: Die Hard, ocena: 4.125915602232819
Film: Rain Man, ocena: 4.071535242958551
Film: The Lord of the Rings: The Return of the King, ocena: 4.020237449257014
Film: A Beautiful Mind, ocena: 4.0151424900648385
Film: Good Will Hunting, ocena: 4.00928080692282
Film: The Lord of the Rings: The Two Towers, ocena: 3.9414763050955943
Film: Indiana Jones and the Last Crusade, ocena: 3.7969764963789236


In [52]:
# najbolj podobni filmi

In [56]:
for row in rp.return_top20_most_sim():
    print(f"Film1: {md.get_title(row[0])}, Film2: {md.get_title(row[1])}, podobnost: {row[2]}")

Film1: The Lord of the Rings: The Return of the King, Film2: The Lord of the Rings: The Two Towers, podobnost: 0.8439842148481421
Film1: The Lord of the Rings: The Two Towers, Film2: The Lord of the Rings: The Return of the King, podobnost: 0.8439842148481421
Film1: The Lord of the Rings: The Fellowship of the Ring, Film2: The Lord of the Rings: The Two Towers, podobnost: 0.8231885401761893
Film1: The Lord of the Rings: The Two Towers, Film2: The Lord of the Rings: The Fellowship of the Ring, podobnost: 0.8231885401761893
Film1: The Lord of the Rings: The Return of the King, Film2: The Lord of the Rings: The Fellowship of the Ring, podobnost: 0.8079374897442505
Film1: The Lord of the Rings: The Fellowship of the Ring, Film2: The Lord of the Rings: The Return of the King, podobnost: 0.8079374897442505
Film1: Kill Bill: Vol. 2, Film2: Kill Bill: Vol. 2, podobnost: 0.7372340224381034
Film1: Kill Bill: Vol. 2, Film2: Kill Bill: Vol. 2, podobnost: 0.7372340224381034
Film1: Star Wars, Film2:

In [57]:
# Priporočanje glede na trenutno ogledano vsebino    

In [19]:
rec_items = rp.similarItems(4993, 10)
print('Filmi podobni "The Lord of the Rings: The Fellowship of the Ring": ')
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))

Filmi podobni "The Lord of the Rings: The Fellowship of the Ring": 
Film: The Lord of the Rings: The Two Towers, ocena: 0.8231885401761893
Film: The Lord of the Rings: The Return of the King, ocena: 0.8079374897442505
Film: Star Wars: Episode V - The Empire Strikes Back, ocena: 0.23961943073496456
Film: Star Wars, ocena: 0.21965586527074096
Film: The Matrix, ocena: 0.21515552706880264
Film: Raiders of the Lost Ark, ocena: 0.19944276706345018
Film: The Usual Suspects, ocena: 0.18321188451910747
Film: Blade Runner, ocena: 0.16399681315410283
Film: Schindler's List, ocena: 0.16105905138148705
Film: Monty Python and the Holy Grail, ocena: 0.15780453798519112
